# FewShotPainAdaptation on Google Colab (T4)\n\nThis notebook installs dependencies, verifies GPU, configures reproducibility, and runs LOSO few-shot training.\n\nBefore running, set Colab runtime to: **GPU (T4)**.

In [ ]:
!nvidia-smi\n!pip -q install -U pip\n# Colab uses Linux GPU runtime; do not install tensorflow-metal here.\n!pip -q install tensorflow==2.18.1 cloudpickle matplotlib numpy pandas scikit-learn scipy seaborn pydantic

In [ ]:
# Option A: clone your repo (recommended). Replace URL if needed.\nREPO_URL = "https://github.com/<your-user>/FewShotPainAdaptation.git"\nPROJECT_DIR = "/content/FewShotPainAdaptation"\n\nimport os\nif not os.path.isdir(PROJECT_DIR):\n    !git clone $REPO_URL $PROJECT_DIR\n%cd $PROJECT_DIR

In [ ]:
import tensorflow as tf\n\ngpus = tf.config.list_physical_devices('GPU')\nprint('Visible GPUs:', gpus)\nif not gpus:\n    raise RuntimeError('No GPU detected. In Colab, set Runtime -> Change runtime type -> GPU.')\n\n# Optional, helps speed on T4.\nfrom tensorflow.keras import mixed_precision\nmixed_precision.set_global_policy('mixed_float16')\nprint('Mixed precision policy:', mixed_precision.global_policy())

In [ ]:
# Point this to a folder containing: X_pre.npy, y_heater.npy, subjects.npy\n# Example if your data is in repo: /content/FewShotPainAdaptation/data\n# Example if in Drive: /content/drive/MyDrive/FewShotPainAdaptation/data\nDATA_DIR = "/content/FewShotPainAdaptation/data"\n\nimport os\nrequired = ['X_pre.npy', 'y_heater.npy', 'subjects.npy']\nmissing = [f for f in required if not os.path.exists(os.path.join(DATA_DIR, f))]\nif missing:\n    raise FileNotFoundError(f'Missing files in DATA_DIR={DATA_DIR}: {missing}')\nprint('Using data directory:', DATA_DIR)

In [ ]:
import json\nimport logging\nimport numpy as np\nfrom data_loaders.pain_ds_config import PainDatasetConfig\nfrom learner.few_shot_pain_learner import FewShotPainLearner\nfrom utils.logger import setup_logger\n\nlogger = setup_logger('FewShotPainLearner', level=logging.INFO)\n\n# Reproducible config\nconfig = PainDatasetConfig(\n    seed=42,\n    deterministic_ops=True,\n    k_shot=3,\n    q_query=3,\n)\n\n# Keep these small first; increase after sanity check.\nNUM_EPOCHS = 3\nEPISODES_PER_EPOCH = 10\nVAL_EPISODES = 5\nFUSION_METHODS = ['attention']\n\nall_results = {}\nfor fusion_method in FUSION_METHODS:\n    logger.info(f'Training with fusion method: {fusion_method}')\n    learner = FewShotPainLearner(\n        config=config,\n        data_dir=DATA_DIR,\n        learning_rate=1e-3,\n        fusion_method=fusion_method,\n        seed=config.seed,\n        deterministic_ops=config.deterministic_ops,\n    )\n    cv_results = learner.train(\n        num_epochs=NUM_EPOCHS,\n        episodes_per_epoch=EPISODES_PER_EPOCH,\n        val_episodes=VAL_EPISODES,\n    )\n    all_results[fusion_method] = cv_results\n\nsummary = {\n    fm: {\n        'avg_test_acc': float(np.mean(res['test_accuracies'])),\n        'std_test_acc': float(np.std(res['test_accuracies'])),\n        'avg_test_loss': float(np.mean(res['test_losses'])),\n    }\n    for fm, res in all_results.items()\n}\nprint(json.dumps(summary, indent=2))\n\nwith open('colab_run_results.json', 'w') as f:\n    json.dump({'summary': summary, 'full': all_results}, f, indent=2)\nprint('Saved results to colab_run_results.json')

## Notes\n- If clone step is not needed (because notebook already lives in repo), just set `PROJECT_DIR` and run `%cd`.\n- For final experiments on T4, scale up `NUM_EPOCHS` and `EPISODES_PER_EPOCH`.\n- Keep `seed` fixed for reproducibility comparisons.